In [1]:
print('OK')

OK


In [2]:
%pwd

'/Users/nguyenhoainam/Desktop/DS-DA/Projects/Medical-Chatbot-Generative-AI/research'

In [3]:
import os
os.chdir("../") # get one folder back

In [4]:
%pwd

'/Users/nguyenhoainam/Desktop/DS-DA/Projects/Medical-Chatbot-Generative-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader # data loading
from langchain.text_splitter import RecursiveCharacterTextSplitter # chunking

In [6]:
# Extract data from the pdf file 

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file(data="Data/")

In [8]:
# extracted_data

In [8]:
# Split data into text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
len(text_chunks)

5860

In [16]:
# text_chunks

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings
embeddings = download_hugging_face_embeddings()

/var/folders/kf/qv7h_2j17gn8fff_ljjtbv_m0000gn/T/ipykernel_52431/2707779312.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/Users/nguyenhoainam/Desktop/DS-DA/Projects/Medical-Chatbot-Generative-AI/my_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))
print(query_result)

Length 384
[-0.034477293491363525, 0.03102315589785576, 0.006734959781169891, 0.02610900066792965, -0.03936200961470604, -0.16030247509479523, 0.06692396849393845, -0.0064414446242153645, -0.047450534999370575, 0.014758884906768799, 0.07087534666061401, 0.05552756041288376, 0.019193310290575027, -0.02625133842229843, -0.010109486989676952, -0.026940500363707542, 0.022307446226477623, -0.02222663350403309, -0.1496926099061966, -0.01749308407306671, 0.007676213979721069, 0.054352302104234695, 0.003254484385251999, 0.031725991517305374, -0.08462141454219818, -0.029405979439616203, 0.05159563943743706, 0.0481240414083004, -0.00331478682346642, -0.05827920511364937, 0.04196926951408386, 0.02221072092652321, 0.128188818693161, -0.022338924929499626, -0.011656248942017555, 0.06292834132909775, -0.03287629410624504, -0.09122605621814728, -0.03117532841861248, 0.052699532359838486, 0.047034818679094315, -0.08420304954051971, -0.030056171119213104, -0.020744822919368744, 0.009517810307443142, -0

In [27]:
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [15]:
from pinecone import Pinecone, ServerlessSpec
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [29]:
# Set Pinecone API as environment variable so that whenever storing the data to the pinecone, 
# pinecone will internally looks for API key inside the environment variable 
# without passing it again and again pc = Pinecone(api_key=PINECONE_API_KEY)

import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [20]:
# store the vector embedding
# Embed each chunk and upsert the embeddings into pinecone index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    embedding=embeddings,
    index_name=index_name,
    documents=text_chunks
)

In [21]:
# Load existing index
 
docsearch = PineconeVectorStore.from_existing_index(
    embedding=embeddings,
    index_name=index_name
)

In [22]:
retriever = docsearch.as_retriever(search_type='similarity', search_kwargs={"k":3}) # 3 relevant answers

In [25]:
retrieved_doc = retriever.invoke("What is Acne?")

In [26]:
retrieved_doc

[Document(id='e2649f50-f25e-4246-819d-0ad91a480f93', metadata={'page': 39.0, 'source': 'Data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='db4b1308-206c-45cf-aeea-7e3f90b4cd42', metadata={'page': 38.0, 'source': 'Data/Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='4e68eb6c-48b5-4791-84a4-dcc439f96ba4', metadata={'page': 37.0, 'source': 'Data/Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when th

In [30]:
# intialize LLM

from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4,max_tokens=500)

In [31]:
# Create the complete chain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [32]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])



Acne is a skin disorder characterized by inflamed sebaceous glands, resulting in pimples on the face, chest, and back. It is caused by clogged pores due to oil, dead skin cells, and bacteria. Acne vulgaris is the most common form of acne, affecting approximately 17 million people in the United States.
